In [1]:
import pandas as pd

# Load the dataset
df = pd.read_csv("../data/Personal_Finance_Dataset.csv")

# Convert 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Add new features
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df['Day_of_Week'] = df['Date'].dt.day_name()
df['Is_Weekend'] = df['Day_of_Week'].isin(['Saturday', 'Sunday'])

# Calculate cumulative monthly expense
df['Cumulative_Monthly_Expense'] = df.groupby([df['Date'].dt.to_period('M')])['Amount'].cumsum()

# Calculate expense category ratio
category_totals = df.groupby('Category')['Amount'].transform('sum')
df['Expense_Category_Ratio'] = df['Amount'] / category_totals

# Display the first few rows
df.head()


,Date,Transaction Description,Category,Amount,Type,Month,Year,Day_of_Week,Is_Weekend,Cumulative_Monthly_Expense,Expense_Category_Ratio
0,2020-01-02,Score each.,Food & Drink,1485.69,Expense,1,2020,Thursday,False,1485.69,0.009315
1,2020-01-02,Quality throughout.,Utilities,1475.58,Expense,1,2020,Thursday,False,2961.27,0.010049
2,2020-01-04,Instead ahead despite measure ago.,Rent,1185.08,Expense,1,2020,Saturday,True,4146.35,0.007312
3,2020-01-05,Information last everything thank serve.,Investment,2291.00,Income,1,2020,Sunday,True,6437.35,0.006307
4,2020-01-13,Future choice whatever from.,Food & Drink,1126.88,Expense,1,2020,Monday,False,7564.23,0.007065


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_csv("../data/Personal_Finance_Dataset.csv")

# Convert 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Add new features
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df['Day_of_Week'] = df['Date'].dt.day_name()
df['Is_Weekend'] = df['Day_of_Week'].isin(['Saturday', 'Sunday'])

# Calculate cumulative monthly expense
df['Cumulative_Monthly_Expense'] = df.groupby([df['Date'].dt.to_period('M')])['Amount'].cumsum()

# Calculate expense category ratio
category_totals = df.groupby('Category')['Amount'].transform('sum')
df['Expense_Category_Ratio'] = df['Amount'] / category_totals

# Step 1: Handle missing values
df_cleaned = df.dropna()

# Step 2: Separate income and expense transactions
df_cleaned['Transaction_Type'] = df_cleaned['Amount'].apply(lambda x: 'Income' if x > 0 else 'Expense')
df_cleaned['Absolute_Amount'] = df_cleaned['Amount'].abs()

# Step 3: Prepare for ML modeling (e.g., monthly aggregation)
monthly_summary = df_cleaned.groupby(['Year', 'Month', 'Transaction_Type'])['Absolute_Amount'].sum().unstack().fillna(0)
monthly_summary['Net_Savings'] = monthly_summary.get('Income', 0) - monthly_summary.get('Expense', 0)

# Step 4: Basic Visualizations
plt.figure(figsize=(10, 6))
sns.barplot(data=df_cleaned[df_cleaned['Transaction_Type'] == 'Expense'], x='Category', y='Absolute_Amount', estimator=sum, ci=None)
plt.xticks(rotation=45)
plt.title('Total Spending by Category')
plt.tight_layout()
plt.savefig("category_spending.png")
plt.close()

monthly_expense = df_cleaned[df_cleaned['Transaction_Type'] == 'Expense'].groupby(df_cleaned['Date'].dt.to_period('M'))['Absolute_Amount'].sum()
monthly_expense.index = monthly_expense.index.to_timestamp()

plt.figure(figsize=(10, 6))
monthly_expense.plot(kind='line', marker='o')
plt.title('Monthly Expense Trend')
plt.ylabel('Total Expense')
plt.xlabel('Month')
plt.grid(True)
plt.tight_layout()
plt.savefig("monthly_expense_trend.png")
plt.close()

# Output the cleaned dataset and summary
df_cleaned.to_csv("../outputs/cleaned_finance_data.csv", index=False)
monthly_summary.to_csv("../outputs/monthly_summary.csv")

print("Data cleaning, preprocessing, and basic visualizations completed.")



C:\Users\Pooja Sugumar\AppData\Local\Temp\ipykernel_24516\1876683258.py:37: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar=None` for the same effect.

  sns.barplot(data=df_cleaned[df_cleaned['Transaction_Type'] == 'Expense'], x='Category', y='Absolute_Amount', estimator=sum, ci=None)


Data cleaning, preprocessing, and basic visualizations completed.
